In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.1 trl==0.4.7

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
torch.cuda.is_available()


In [7]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [1]:
import os
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import io
from huggingface_hub import notebook_login
import pandas as pd

ModuleNotFoundError: No module named 'peft'

In [ ]:
notebook_login()

In [ ]:

df = pd.read_excel('/kaggle/input/fine-tuning/Dataset_creation.xlsx')
df["formatted_instruction"] = df.apply(lambda x: f"<s>[INST] {x['Instructions']} [/INST] {x['Responses']} </s>", axis=1)
#df = df.drop(columns=df.columns[:2])
df = df.drop(columns=['Instructions', 'Responses'])
df.to_csv("conversational_genAI.csv", index=False)
#created_dataset = Dataset.from_pandas(df)
#print(created_dataset)


df_read = pd.read_csv("/kaggle/working/conversational_genAI.csv")

# Ensure the key 'formatted_instruction' exists in the DataFrame
if "formatted_instruction" not in df_read.columns:
    raise ValueError("The key 'formatted_instruction' does not exist in the DataFrame")

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_read)

# Create a DatasetDict with the train split
dataset_dict = DatasetDict({
    "train": dataset
})
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model name
model_name = "meta-llama/Llama-2-7b-hf"
new_model = "llama-2-7b-user-manuals"

max_seq_length = 128                    # Maximum sequence length to use
packing = False                        # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0}                   # Load the entire model on the GPU 0


# Tokenize the dataset (example tokenization step)
def tokenize_function(examples):
    return tokenizer(examples["formatted_instruction"], padding="max_length", truncation=True, max_length=max_seq_length)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)

#dataset_name = "/content/conversational_genAI.csv"
#def tokenize(example):
#    return tokenizer(example['formatted_instruction'], truncation=True, padding='max_length',max_length=max_seq_length)


#tokenized_dataset = created_dataset.map(tokenize, batched=True, remove_columns=created_dataset.column_names)
#print(tokenized_dataset[0])

In [ ]:
#-----QLora parameters------#
lora_r = 16            # attention mechanism
lora_alpha = 64        # scaling purpose
lora_dropout = 0.1     # dropout probaility layer

#----Bits and bytes parameters-------#
load_4bit=True                      # Activate 4-bit precision base model loading
bnb_4bit_quant_type="nf4"           # Quantization type (fp4 or nf4)
bnb_4bit_compute_dtype=torch.float16    # Compute dtype for 4-bit base models
#compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
use_nested_quant = False

#----Training arguments parameters-----#
output_dir = "./results"               # output directory
num_train_epochs = 100                   # traning epochs
fp16 = False                           # Enable fp16/bf16 training (set bf16 to True with an A100)
bf16 = False
per_device_train_batch_size = 4        # Batch size per GPU for training
per_device_eval_batch_size = 16         # Batch size per GPU for evaluation
gradient_accumulation_steps = 2        # Number of update steps to accumulate the gradients for
gradient_checkpointing = True
max_grad_norm = 0.3                    # Maximum gradient normal (gradient clipping)
learning_rate = 2e-4
weight_decay = 0.01                   # Weight decay to apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit"            # optimizer
lr_scheduler_type = "cosine"           # Learning rate scheduler
max_steps = -1                         # Number of training steps (overrides num_train_epochs)
warmup_ratio = 0.03                    # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = True                 # Group sequences into batches with same length. Saves memory and speeds up training considerably
save_steps = 170                         # Save checkpoint every X updates steps
logging_steps = 17                     # Log every X updates steps




In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# df_read = pd.read_csv("/content/conversational_genAI.csv")
# Ensure the key exists in the DataFrame
if "formatted_instruction" not in df_read.columns:
    raise ValueError("The key 'formatted_instruction' does not exist in the DataFrame")

# Assuming you want to create a DatasetDict with the key "formatted_instruction"
dataset_dict = DatasetDict({"formatted_instruction": dataset})
# Convert DataFrame to Hugging Face Dataset
#dataset = Dataset.from_pandas(df_read)
#print(type(dataset))

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    peft_config=peft_config,
    dataset_text_field="formatted_instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(new_model)



In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What are the parameters for pulse time in pulsed operation?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])